In [1]:
import numpy as np     
import networkx as nx 
import nltk 
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance

Reference: https://stackabuse.com/text-summarization-with-nltk-in-python/

https://www.fireblazeaischool.in/blogs/text-summarization-using-nlp/

In [2]:
stop_words = stopwords.words('english')
sample_input = 'Junk foods taste good that’s why it is mostly liked by everyone of any age group especially kids and school going children. They generally ask for the junk food daily because they have been trend so by their parents from the childhood. They never have been discussed by their parents about the harmful effects of junk foods over health. According to the research by scientists, it has been found that junk foods have negative effects on the health in many ways. They are generally fried food found in the market in the packets. They become high in calories, high in cholesterol, low in healthy nutrients, high in sodium mineral, high in sugar, starch, unhealthy fat, lack of protein and lack of dietary fibers. Processed and junk foods are the means of rapid and unhealthy weight gain and negatively impact the whole body throughout the life. It makes able a person to gain excessive weight which is called as obesity. Junk foods tastes good and looks good however do not fulfil the healthy calorie requirement of the body. Some of the foods like french fries, fried foods, pizza, burgers, candy, soft drinks, baked goods, ice cream, cookies, etc are the example of high-sugar and high-fat containing foods. It is found according to the Centres for Disease Control and Prevention that Kids and children eating junk food are more prone to the type-2 diabetes. In type-2 diabetes our body become unable to regulate blood sugar level. Risk of getting this disease is increasing as one become more obese or overweight. It increases the risk of kidney failure. Eating junk food daily lead us to the nutritional deficiencies in the body because it is lack of essential nutrients, vitamins, iron, minerals and dietary fibers. It increases risk of cardiovascular diseases because it is rich in saturated fat, sodium and bad cholesterol. High sodium and bad cholesterol diet increases blood pressure and overloads the heart functioning. One who like junk food develop more risk to put on extra weight and become fatter and unhealthier. Junk foods contain high level carbohydrate which spike blood sugar level and make person more lethargic, sleepy and less active and alert. Reflexes and senses of the people eating this food become dull day by day thus they live more sedentary life. Junk foods are the source of constipation and other disease like diabetes, heart ailments, clogged arteries, heart attack, strokes, etc because of being poor in nutrition. Junk food is the easiest way to gain unhealthy weight. The amount of fats and sugar in the food makes you gain weight rapidly. However, this is not a healthy weight. It is more of fats and cholesterol which will have a harmful impact on your health. Junk food is also one of the main reasons for the increase in obesity nowadays.This food only looks and tastes good, other than that, it has no positive points. The amount of calorie your body requires to stay fit is not fulfilled by this food. For instance, foods like French fries, burgers, candy, and cookies, all have high amounts of sugar and fats. Therefore, this can result in long-term illnesses like diabetes and high blood pressure. This may also result in kidney failure. Above all, you can get various nutritional deficiencies when you don’t consume the essential nutrients, vitamins, minerals and more. You become prone to cardiovascular diseases due to the consumption of bad cholesterol and fat plus sodium. In other words, all this interferes with the functioning of your heart. Furthermore, junk food contains a higher level of carbohydrates. It will instantly spike your blood sugar levels. This will result in lethargy, inactiveness, and sleepiness. A person reflex becomes dull overtime and they lead an inactive life. To make things worse, junk food also clogs your arteries and increases the risk of a heart attack. Therefore, it must be avoided at the first instance to save your life from becoming ruined.The main problem with junk food is that people don’t realize its ill effects now. When the time comes, it is too late. Most importantly, the issue is that it does not impact you instantly. It works on your overtime; you will face the consequences sooner or later. Thus, it is better to stop now.You can avoid junk food by encouraging your children from an early age to eat green vegetables. Their taste buds must be developed as such that they find healthy food tasty. Moreover, try to mix things up. Do not serve the same green vegetable daily in the same style. Incorporate different types of healthy food in their diet following different recipes. This will help them to try foods at home rather than being attracted to junk food.In short, do not deprive them completely of it as that will not help. Children will find one way or the other to have it. Make sure you give them junk food in limited quantities and at healthy periods of time. '

In [9]:
# # Abstractive Text Summarizer
# from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig

# # Loading the model and tokenizer for bart-large-cnn
# tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
# model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# In [2]:

# # Encoding the inputs and passing them to model.generate()
# inputs = tokenizer.batch_encode_plus([sample_input],return_tensors='pt')
# summary_ids = model.generate(inputs['input_ids'], early_stopping=True)
# bart_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
# print(bart_summary)

ImportError: 
BartForConditionalGeneration requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.


In [4]:
# Summarize using Cosine Similarity

class cosine_summary:
    
    def extract_vector(self, sentence, all_words, stop_words):
        
        extracted_vector = [0] * len(all_words)
        for word in sentence:
            if word in stop_words:
                continue
            extracted_vector[all_words.index(word)] += 1
        return extracted_vector
    
    def sentence_similarity(self, s1, s2, stop_words):

        s1 = [word.lower() for word in s1]
        s2 = [word.lower() for word in s2]

        all_words = list(set(s1 + s2))

        v1 = self.extract_vector(s1, all_words, stop_words)
        v2 = self.extract_vector(s2, all_words, stop_words)

        return 1 - cosine_distance(v1, v2)
    
    
    def summarise_text(self, ranked_sentences, threshold):
        summary = []
        max_len = len(ranked_sentences)

        for i in range(max_len):
            if(ranked_sentences[i][0]< 0.02):
                break
            summary.append(" ".join(ranked_sentences[i][1]))

        summary = ". ".join(summary)

        return summary
    
    
    
    def summariser(self, text):
        #Breaking Down text to Sentences
        sentences = []
        
        #STOP Words
        stop_words = stopwords.words('english')
        split_text = text.split(". ")
        
        for sentence in split_text:
            sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
        
        sentences.pop()
        
        #Similarity Matrix Building
        similarity_matrix = np.zeros((len(sentences), len(sentences)))

        for i1, sentence1 in enumerate(sentences):
            for i2, sentence2 in enumerate(sentences):
                if sentence1 == sentence2:
                    similarity_matrix[i1][i2] = 0
                    continue
                similarity_matrix[i1][i2] = self.sentence_similarity(sentence1, sentence2, stop_words)
                
        #Ranking sentences
        similarity_network = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(similarity_network)
        
        # Sorting the Sentences with scores in descending order
        i = 0
        ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentences)), reverse=True)
        
        summary = self.summarise_text(ranked_sentences, 10)
        
        return summary, ranked_sentences

In [5]:
summ = cosine_summary()
summerised_text, ranked_sentences = summ.summariser(sample_input)

In [6]:
len(summerised_text)

2613

In [7]:
len(sample_input)

4870

In [8]:
summerised_text

'Junk food is the easiest way to gain unhealthy weight. One who like junk food develop more risk to put on extra weight and become fatter and unhealthier. To make things worse, junk food also clogs your arteries and increases the risk of a heart attack. Make sure you give them junk food in limited quantities and at healthy periods of time. Junk food is also one of the main reasons for the increase in obesity nowadays.This food only looks and tastes good, other than that, it has no positive points. Eating junk food daily lead us to the nutritional deficiencies in the body because it is lack of essential nutrients, vitamins, iron, minerals and dietary fibers. They generally ask for the junk food daily because they have been trend so by their parents from the childhood. Furthermore, junk food contains a higher level of carbohydrates. It is found according to the Centres for Disease Control and Prevention that Kids and children eating junk food are more prone to the type-2 diabetes. Proces